In [166]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4
import sys

class WRFPlay(object):
    dataset = ""
    run = False
    LATINDEX = ""  # This is the south_north dimension in the NetCDF file
    LONINDEX = ""  # This is the west-east dimension in the NetCDF File
    QVAPOR = ""    # 3D mositure - QVAPOR
    P = ""         # the perturbation pressure
    PB = ""
    T = ""
    T_c = ""
    emessage=""    # Temperature in celcius
    def __init__(self, ncfilepath):
        try:
            #verify file exists & all the variables needed exist with the given file
            #self.dataset = netCDF4.Dataset(ncfilepath, 'r')
            self.verifyNetCDF4File(ncfilepath)
        except Exception as error:
            print(error)
        else:
            self.run = True
         
        
    def sounding_plot(self, latindex=1000, lonindex=1000):
        if self.run:
            try:
                                
                #verify lat, long falls in the range
                self.verifyLatLong(latindex,lonindex)
                             
                
                # Create the total pressure millibar field
                P_mb = (self.P + self.PB) * 0.01

                # let's get the 3d array of temperature in degrees C
                theta = self.T + 300.0 # convert petrubation potential temperature in Kelvin
                T_c = theta*((P_mb)/1000.0)**(2.0/7.0) -273.0

#                 # let's get the dew point
#                 # As per the formula dewpoint T = B/ln(X) where X = Ae/wp
#                 A = 2.53 * (10**9) # coversion from 1 kPa = 10 mbar
#                 B = 5.42 * (10**3)
#                 e = 0.622
#                 w = self.QVAPOR
#                 p = P_mb  

#                 x= (A*e)/(w*p)
#                 T_d = B / (np.log(0.622 * 2.53 * (10**9)/np.multiply(self.QVAPOR,P_mb)))
#                 T_d = T_d - 273.15 #convert to celsius

                T_d = self.computeTD(P_mb)

                #Vertical plot
                t_sounding = T_c[0,:,self.LATINDEX,self.LONINDEX]
                t_d_sounding = T_d[0,:,self.LATINDEX,self.LONINDEX]
                p_sounding = P_mb[0,:,self.LATINDEX,self.LONINDEX]
                plt.semilogy(t_sounding, p_sounding,  label='T')
                plt.semilogy(t_d_sounding, p_sounding, color='green', label='Td')
                plt.legend(loc='upper right')
                plt.title('Lat/Lon indices: ' + str(self.LATINDEX) +'/' + str(self.LONINDEX))
                plt.ylim(ymin=50.0, ymax=1050.0)
                plt.gca().invert_yaxis()

                #Set up ticks and labels on the y-axis
                ylocations = np.arange(100,1000,100)
                plt.yticks(ylocations, ylocations)

                plt.show()
                self.emessage= ""
            except Exception as error:
                print(error)
            
                
        else:
            print()
    def close(self):
        if self.run:
            self.dataset.close()
            
    def verifyLatLong(self, lat, lon):
        if lat == 1000:
            raise Exception('Missing latindex argument')
        elif lon == 1000:
            raise Exception('Missing lonindex argument')
        self.LATINDEX = lat 
        self.LONINDEX= lon
    #method to verify NetCDF4 supplied by the user is valid
    def verifyNetCDF4File(self, file):
        
        
        try:
            #verify file exists & all the variables needed exist with the given file
            self.dataset = netCDF4.Dataset(file, 'r')
            self.run = True
        except Exception as error:
            self.emessage += "Not a valid  netCDF4 file "
            self.run = False
        if self.dataset:
            
            try:
                #verify the variables in file
                varkeys = self.dataset.variables.keys()
            except Exception as error:
                self.emessage += " Variables are not defined in the file \n"
                self.run = False

            try:
                #verify the variables in file
                dimkeys = self.dataset.dimensions.keys()

            except Exception as error:
                self.emessage += " Dimensions are not defined in the file \n"
                self.run = False
            
            try:
                self.QVAPOR = self.verifyVar("QVAPOR", "Water Vapor mixing ratio 'QVAPOR'", "kg kg-1")
            except Exception as error:
                self.run = False
            try:
                self.P = self.verifyVar("P", "Perturbation pressure 'P'", "Pa")
            except Exception as error:
                self.run = False
            try:
                self.PB = self.verifyVar("PB", "Base STate Pressure 'PB'", "Pa")
            except Exception as error:
                self.run = False
            try:
                self.T = self.verifyVar("T", "Potential temperature 'T'", "K")
            except Exception as error:
                self.run = False
            
       
        if self.emessage:
            raise Exception(self.emessage)
            
    #verify variables
    def verifyVar(self, variable, description, units):
        var = None
        try:
            var = self.dataset.variables[variable]
            varNpArray = var[:]
        except:
            self.emessage = self.emessage + description + " variable is not accessible \n"
            self.run = false
        try:
            if units!=var.units:
                raise Exception()
            
        except:
            self.emessage = self.emessage + description + " variable units are not " +var.units +" \n"
            self.run =false
        return varNpArray
    
    #compute Td (Temperature dewpoint)
    def computeTD(self, pressure)
        # Create the total pressure millibar field
        P_mb = (self.P + self.PB) * 0.01
            
        # let's get the dew point
        # As per the formula dewpoint T = B/ln(X) where X = Ae/wp
        A = 2.53 * (10**9) # coversion from 1 kPa = 10 mbar
        B = 5.42 * (10**3)
        e = 0.622
        w = self.QVAPOR
        p = pressure

        x= (A*e)/(w*p)
        T_d = B / (np.log(x)
        T_d = T_d - 273.15 #convert to celsius
        return T_d

    
WRFOUT_FILE_PATH = 'wrfout_d01_2017-09-09_12:00:00'
W = WRFPlay(ncfilepath=WRFOUT_FILE_PATH)
W.sounding_plot(lonindex=96, latindex=68)
W.sounding_plot(latindex=0)
W.close()



SyntaxError: invalid syntax (<ipython-input-166-61ae802cdf77>, line 159)